In [2]:
from flask import Flask, render_template, flash, request, redirect, send_from_directory, url_for, session, jsonify
from wtforms import Form, TextField, TextAreaField, validators, StringField, SubmitField,widgets, SelectMultipleField
from flask_wtf import FlaskForm
import pandas as pd
import numpy as np
import os
import sys
import joblib
import json
import matplotlib.pyplot as plt
%matplotlib inline

def getsym(data, s):
    x = pd.Series((data[data["Disease_name"]==s].values==1)[0],index=data.columns)
    return data.loc[data[data["Disease_name"]==s].index,x].columns


__file__ = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__),'')))

DIR = os.path.dirname(os.path.abspath(__file__))
temp_dir = os.path.join(os.path.abspath(__file__),'templates')
stat_dir = os.path.join(os.path.abspath(__file__),'static')

# Secret key for server
SECRET_KEY = os.urandom(32)
#app.config['SECRET_KEY'] = SECRET_KEY

app = Flask(__name__, template_folder = temp_dir, static_folder=stat_dir)
app.config['SECRET_KEY'] = SECRET_KEY
#app.config.from_object(__name__)
@app.route("/", defaults={'path',''}, methods=["GET", "POST"])
@app.route("/<path:path>", methods=["GET", "POST"])
def index_my(path):
    dis_sym = pd.read_csv('data/Training.csv')
    cols = dis_sym.columns[1:].tolist()
    cols = json.dumps(cols)
    return render_template('index.html', cols=cols)

@app.route("/predict",methods=['POST'])
def predict():
    if request.method == 'POST' :
        #symptoms = 'Symptoms'
        x = request.get_data().decode("utf-8")
        
        search = json.loads(x)['data']
        print(search)
        dis_sym = pd.read_csv('data/Training.csv')
        cols = dis_sym.columns[1:]
        #header("ContentType: application/json")

        clf = joblib.load('data/MES.pkl') # Load Model
        
        ## Feature Importances
        
        importances = clf.feature_importances_
        indices = np.argsort(importances)[::-1]

        features = cols
        
        ## Features sorted
        
        Fea_Dict = {}
        for i,f in enumerate(features):
              Fea_Dict[f] = i            # Features Dictionary
                
        ## Disease Prediction
        
        #search = ['Diarrhea','Nausea','Vomiting','Chills','Fever','Headache']

        sample = np.zeros((len(features),), dtype=np.int)

        for i,s in enumerate(search):
              sample[Fea_Dict[s]] = 1

        sample_in = np.array(sample).reshape(1,len(sample))

        dis_pred_prob = pd.DataFrame(clf.predict_proba(sample_in), columns=clf.classes_)
        dis_pred_prob = dis_pred_prob[dis_pred_prob>0.0].dropna(axis=1)
        dis_pred_prob = dis_pred_prob.sort_values(axis=1, by= 0, ascending= False)
        #print(dis_pred_prob)
        
        d = dis_pred_prob.to_dict('split')
        d = dict((i,d[i]) for i in d.keys() if i!='index')
        d["dis_sym"] = dict((i,getsym(dis_sym,i).tolist()) for i in d["columns"])
        #print(d)
        
        #print(x[0]['data'])
        #print(request.form.get('default'))
        #model = joblib.load('../data/all_mnb.pkl')
    #print(symptoms)
    return jsonify(d)

    
    
if __name__ == "__main__":
    app.run(host = '0.0.0.0', port = 5001, debug=True, use_reloader = False)

 * Serving Flask app "My_MES_Project" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2020 19:56:21] "GET /My_MES_Project HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 19:56:21] "GET /static/images/favicon.ico HTTP/1.1" 200 -
